## This script is used to download the NAIP images automatically
This script will download the Natural color NAIP images and the false color NAIP images. The script will download the NAIP from a database maintained by Microsoft. We need to download NAIP tiles first before doing any raster based operations, like, mosaci, clip, mask, etc.

Modifed by Xiaojiang Li, Temple University on Feb 11, 2022

Last updated by Xiaojiang Li, Temple University, Feb 11, 2024

In [1]:
## This script is the updated version for the NAIP image downloading
## The previous version has been deprecated, since Microsoft Updated
## the data storage on Azure and transfered to the Planertory Computer
## Last modified by Xiaojiang Li, Temple University, Biometeors June 2, 2022

## https://planetarycomputer.microsoft.com/dataset/naip#Example-Notebook

import os, os.path
from pystac_client import Client
import planetary_computer as pc
import rioxarray
import rasterio as rio
import fiona
import pyproj
from shapely.geometry import shape, Point, Polygon, mapping
from functools import partial
from shapely.ops import transform
import urllib.request
import numpy as np
import geopandas as gpd
import pyproj


# Set the environment variable PC_SDK_SUBSCRIPTION_KEY, or set it here.
# The Hub sets PC_SDK_SUBSCRIPTION_KEY automatically.
pc.settings.set_subscription_key("st=2022-06-01T22%3A55%3A32Z&se=2022-06-02T23%3A40%3A32Z&sp=rl&sv=2020-06-12&sr=c&skoid=c85c15d6-d1ae-42d4-af60-e2ca0f81359b&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2022-06-02T20%3A23%3A33Z&ske=2022-06-09T20%3A23%3A33Z&sks=b&skv=2020-06-12&sig=U5e1SoZGZqIcunum5BKWcmsVi2bmS1tBsr94LPI%2BLoc%3D")


In [14]:
def naip_downloader(shpfile, outfolder, startDate, endDate):
    '''This function will use the shapefile as the limit and find the 
    tiles that overlap with the shapefile and then download those tiles
    
    Parameters:
        shpfile: the shapefile of the city limit
        outfolder: the output folder of the NAIP images
        startDate: the start date to check the NAIP inventory, year-month-day, 2017-06-01/2017-09-30
        endDate: the end date to check the available NAIP year-month-day, 2017-06-01/2017-09-30
    '''
    
    import fiona
    
    lyr = fiona.open(shpfile)
    for feat in lyr:
        area_of_interest_city = feat['geometry']
        # print(area_of_interest)
    
    left, bottom, right, top = lyr.bounds
    area_of_interest = {
        "type": "Polygon",
        "coordinates": [
            [
                [left, bottom],
                [right, bottom],
                [right, top],
                [left, top],
                [left, bottom],
            ]
        ],
    }
    
    print(area_of_interest)
    
    range_time = startDate + '/' + endDate
    print(range_time)
    
    catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    
    search_tiles = catalog.search(
        collections=["naip"], intersects=area_of_interest, datetime=range_time
    )
    items_tiles = list(search_tiles.get_items())
    print(f"{len(items_tiles)} Items found in the range %s"%(range_time))
    print(items_tiles)
    
    ## loop all the intersected tiles and download them all
    for item in items_tiles:
        # href = pc.sign(item.assets["image"].href)
        href = item.assets["image"].href
        
        outfilename = os.path.join(outfolder, item.id + ".tif")
        urllib.request.urlretrieve(href, outfilename)

        ### check if the NAIP tile intersects with the shapefile, if not remove it.
        lu_dataset = rio.open(outfilename)
        epsgcode = lu_dataset.crs.to_epsg()
        lu_bounds = lu_dataset.bounds
        lu_box = Polygon([(lu_bounds.left, lu_bounds.bottom), \
                            (lu_bounds.left, lu_bounds.top), \
                            (lu_bounds.right, lu_bounds.top), \
                            (lu_bounds.right, lu_bounds.bottom)])

        city_geom = shape(area_of_interest_city)
        print('city_geom is:', city_geom)
        print("The lubox is:", lu_box)
        
        project = pyproj.Transformer.from_proj(
            pyproj.Proj('epsg:4326'), # source coordinate system
            pyproj.Proj('epsg:%s'%(epsgcode)), 
            always_xy=True) # destination coordinate system
        city_geom_proj = transform(project.transform, city_geom)  # apply projection
        
        ## make sure the projections are consistent
        if city_geom_proj.intersects(lu_box):
            print("intersected :", outfilename)
            r = lu_dataset.read(1) ## start from 1, 2, 3, 4
            g = lu_dataset.read(2)
            b = lu_dataset.read(3)
            ir = lu_dataset.read(4)
            #rgb = np.dstack((r,g,b))
            cir = np.stack((ir,r,g,b))
            
            out_meta = lu_dataset.meta.copy()
            out_meta.update({'count': 4})
            # os.remove(outfilename)
            with rio.open(outfilename, "w", **out_meta) as dest:
                dest.write(cir)
        else:
            print("The CIR is not intersected, remove it=--------")
            os.remove(outfilename)
        

In [ ]:
# import geopandas as gpd
# df = gpd.read_file('data/city_limit.shp')
# df_proj = df.to_crs(epsg=4326)
# df_proj.to_file('data/city_limit_4326.shp', driver='ESRI Shapefile')


In [15]:
import os, os.path
import argparse
import geopandas as gpd

# make sure the project is wgs84, 4326
shapefile = 'data/city_limit_4326.shp'

# using the reprojected geom shape to find the corresponding tile
cir_naips = 'cir-naip'
if not os.path.exists(cir_naips): os.mkdir(cir_naips)

# check on GEE, https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ
startDate = '2017-06-01'
endDate = '2017-09-30'
naip_downloader(shapefile, cir_naips, startDate, endDate)


{'type': 'Polygon', 'coordinates': [[[-75.28030313034645, 39.867465570687145], [-74.9557457320632, 39.867465570687145], [-74.9557457320632, 40.137927528193686], [-75.28030313034645, 40.137927528193686], [-75.28030313034645, 39.867465570687145]]]}
2017-06-01/2017-09-30


/Users/tul69181/anaconda3/envs/geospatial/lib/python3.10/site-packages/pystac_client/item_search.py:835: FutureWarning: get_items() is deprecated, use items() instead
  warnings.warn(


46 Items found in the range 2017-06-01/2017-09-30
[<Item id=pa_m_4007562_se_18_1_20170910_20171207>, <Item id=pa_m_4007562_ne_18_1_20170910_20171207>, <Item id=pa_m_4007554_se_18_1_20170910_20171207>, <Item id=nj_m_4007564_se_18_1_20170824_20171102>, <Item id=nj_m_4007457_sw_18_1_20170824_20171102>, <Item id=nj_m_4007457_nw_18_1_20170824_20171102>, <Item id=nj_m_3907516_nw_18_1_20170809_20171102>, <Item id=nj_m_3907515_ne_18_1_20170809_20171102>, <Item id=nj_m_3907508_sw_18_1_20170809_20171102>, <Item id=nj_m_3907508_nw_18_1_20170809_20171102>, <Item id=nj_m_3907507_se_18_1_20170809_20171102>, <Item id=nj_m_3907507_ne_18_1_20170809_20171102>, <Item id=pa_m_4007556_se_18_1_20170801_20171207>, <Item id=nj_m_3907516_ne_18_1_20170731_20171102>, <Item id=nj_m_3907508_se_18_1_20170731_20171102>, <Item id=nj_m_3907508_ne_18_1_20170731_20171102>, <Item id=nj_m_3907409_nw_18_1_20170731_20171102>, <Item id=nj_m_3907401_sw_18_1_20170731_20171102>, <Item id=nj_m_3907401_nw_18_1_20170731_20171102>,

KeyboardInterrupt: 